In [ ]:
from google.colab import drive
drive.mount(‘/content/gdrive’)

In [ ]:
!pip install diffusers torch Pillow
!pip install xformers  # Optional for memory-efficient attention

In [ ]:
from pathlib import Path
import os
from PIL import ImageOps
import torch
from diffusers import DPMSolverMultistepScheduler, StableDiffusionPipeline

# Set CUDA device (if available)
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

def generate_and_save_mammograms(vendor, view, density, area, laterality, priority, negative_prompt, guidance_scale, diffusion_steps, seed_checkbox, seed, num_samples=500):
    """Function to generate and save 500 mammogram images."""

    # Define model and load weights
    model_dir = 'Likalto4/fusion_healthy'  # Replace with your model's path

    # Load the Stable Diffusion pipeline
    pipe = StableDiffusionPipeline.from_pretrained(
        model_dir,
        safety_checker=None,
        torch_dtype=torch.float16
    ).to("cuda")

    # Use DPMSolverMultistepScheduler
    pipe.scheduler = DPMSolverMultistepScheduler.from_config(pipe.scheduler.config)
    pipe.enable_xformers_memory_efficient_attention()

    # Define the output directory
    output_dir = '/content/drive/MyDrive/new'
    Path(output_dir).mkdir(parents=True, exist_ok=True)  # Ensure the directory exists

    # Build prompt based on priority
    if priority == 'area':
        prompt = f'a {vendor} mammogram in {view} view with {area} area and {density} density'
    elif priority == 'density':
        prompt = f'a {vendor} mammogram in {view} view with {density} density and {area} area'
    elif priority == 'only area':
        prompt = f'a {vendor} mammogram in {view} view with {area} area'
    elif priority == 'only density':
        prompt = f'a {vendor} mammogram in {view} view with {density} density'
    else:
        raise ValueError('Invalid priority value')

    # Set seed for reproducibility if needed
    if seed_checkbox:
        generator = torch.Generator(device='cuda')
        generator.manual_seed(seed)
    else:
        generator = None

    # Loop to generate images
    for i in range(num_samples):
        # Generate image
        with torch.autocast("cuda"), torch.inference_mode():
            image = pipe(
                prompt=prompt,
                negative_prompt=negative_prompt,
                num_images_per_prompt=1,
                num_inference_steps=diffusion_steps,
                guidance_scale=guidance_scale,
                height=512,
                width=512,
                generator=generator
            ).images

        # Get the output image and handle laterality
        pil_output = image[0]
        if laterality == 'R':
            pil_output = ImageOps.mirror(pil_output)

        # Save the image with a unique filename
        output_path = os.path.join(output_dir, f"mammogram_{i+1}.png")
        pil_output.save(output_path)

        print(f"Saved image {i+1}/{num_samples} to {output_path}")

if __name__ == '__main__':
    # Set your parameters for generating images
    vendor = 'Siemens'
    view = 'MLO'
    density = 'very high'
    area = 'small'
    laterality = 'L'
    priority = 'area'
    negative_prompt = ''
    guidance_scale = 6
    diffusion_steps = 24
    seed_checkbox = True
    seed = 1338
    num_samples = 500  # Number of images to generate

    # Call the function to generate and save images
    generate_and_save_mammograms(vendor, view, density, area, laterality, priority, negative_prompt, guidance_scale, diffusion_steps, seed_checkbox, seed, num_samples)